In [1]:
import requests # Python 2.7, requests version 2.12.4
import pandas as pd          # pandas version 0.18.1

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/M.DE.PCPI_PC_CP_A_PT' # adjust as needed
data = requests.get('{}{}'.format(url,key)).json()
print data['CompactData']['DataSet']['Series']['Obs'][-1]
# Navigate to observations in JSON data
obs = data['CompactData']['DataSet']['Series']['Obs']

{u'@TIME_PERIOD': u'2017-02', u'@OBS_VALUE': u'2.15962441314554'}


In [2]:
# Create pandas dataframe from the observations
df = pd.DataFrame(obs)[['@OBS_VALUE','@TIME_PERIOD']]
df.columns = ['value', 'date']
df.loc[:,'value'] = df['value'].astype(float).round(1)
df = df.set_index(pd.to_datetime(df['date']))['value']

In [8]:
lvalue = df[-1]
ldate = str(df.index[-1].strftime('%b %Y'))
onemo = round(lvalue - df[-2], 2)
onemoar = '<i class="fa fa-caret-up" aria-hidden="true"></i>'
if onemo < -0.01:
    onemoar = '<i class="fa fa-caret-down" aria-hidden="true"></i>'
latest = ' Latest: {}: {}% <br>  One month change: {} {}'.format(ldate, lvalue, onemoar, onemo)

In [4]:
from datetime import datetime as dt
import time
xlabel = time.mktime(dt(1995, 2, 1, 2, 0, 0).timetuple())*1000

In [5]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Span, Label, HoverTool
from bokeh.embed import components

source = ColumnDataSource({'x': df.index, 'y': df.values, 'Date': df.index.strftime('%b %Y')})
tooltips = """
            <div>
                <span style="font-size: 12px;">@Date:</span>
                <span style="font-size: 12px; font-weight: bold;">@y{1.1}</span>
            </div>
"""
# horizontal line at zero
hline = Span(location=0, dimension='width', line_color='gray', line_width=1)
p = figure(width=700,height=300,x_axis_type='datetime',
           tools=['pan,wheel_zoom,box_zoom,reset'], logo=None, 
           toolbar_location="below", toolbar_sticky=False)
p.add_layout(hline)
p.sizing_mode = 'scale_width'
p.line('x', 'y', source = source, line_width=3, color='Orange', alpha=1.0, level='overlay')
p.xgrid.grid_line_color = None
p.outline_line_color = 'white'
p.axis.axis_line_color = 'white'
citation = Label(x=xlabel, y=5.5, render_mode='css', x_units='data', text_font_size='9pt', 
                 text_color='#686868', text=latest, background_fill_color='white', background_fill_alpha=1.0)
p.add_layout(citation)
p.add_tools(HoverTool(tooltips=tooltips, line_policy='nearest', show_arrow=False))
p.toolbar.active_drag = None
script, div = components(p)

In [6]:
with open('C:/Working/bdecon.github.io/plots/de_cpi.html', 'w') as text_file:
    text_file.write(                  # txt file created
        '{} {}'.format(script, div)
    )